<a href="https://colab.research.google.com/github/Moizzzzzzzzzz/Movie_recommendation_system/blob/main/Movies_Recommended_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Adjust the path below if your file is in a different location within Google Drive
df = pd.read_csv('/content/drive/MyDrive/movies_metadata.csv')



/tmp/ipython-input-3201431658.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/movies_metadata.csv')


In [4]:
df.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
df.isnull().sum()

,0
adult,0
belongs_to_collection,40972
budget,0
genres,0
homepage,37684
id,0
imdb_id,17
original_language,11
original_title,0
overview,954


In [7]:
movies_df = df[['id', 'title', 'genres', 'overview', 'vote_average', 'vote_count', 'tagline', 'spoken_languages']].copy()
display(movies_df.head())

,id,title,genres,overview,vote_average,vote_count,tagline,spoken_languages
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",7.7,5415.0,NaN,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,6.9,2413.0,Roll the dice and unleash the excitement!,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,6.5,92.0,Still Yelling. Still Fighting. Still Ready for...,"[{'iso_639_1': 'en', 'name': 'English'}]"
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",6.1,34.0,Friends are the people who let you be yourself...,"[{'iso_639_1': 'en', 'name': 'English'}]"
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,5.7,173.0,Just When His World Is Back To Normal... He's ...,"[{'iso_639_1': 'en', 'name': 'English'}]"


In [9]:
import ast

def extract_names(column_string):
    if pd.isna(column_string):
        return ''
    try:
        # Safely evaluate the string as a Python literal
        list_of_dicts = ast.literal_eval(column_string)
        # Extract 'name' from each dictionary
        names = [d['name'] for d in list_of_dicts if 'name' in d]
        return ', '.join(names)
    except (ValueError, SyntaxError):
        # Handle cases where the string is not a valid list of dictionaries
        return ''

# Apply the function to the 'genres' column
movies_df['genres'] = movies_df['genres'].apply(extract_names)

# Apply the function to the 'spoken_languages' column
movies_df['spoken_languages'] = movies_df['spoken_languages'].apply(extract_names)

display(movies_df.head())

,id,title,genres,overview,vote_average,vote_count,tagline,spoken_languages
0,862,Toy Story,,"Led by Woody, Andy's toys live happily in his ...",7.7,5415.0,NaN,
1,8844,Jumanji,,When siblings Judy and Peter discover an encha...,6.9,2413.0,Roll the dice and unleash the excitement!,
2,15602,Grumpier Old Men,,A family wedding reignites the ancient feud be...,6.5,92.0,Still Yelling. Still Fighting. Still Ready for...,
3,31357,Waiting to Exhale,,"Cheated on, mistreated and stepped on, the wom...",6.1,34.0,Friends are the people who let you be yourself...,
4,11862,Father of the Bride Part II,,Just when George Banks has recovered from his ...,5.7,173.0,Just When His World Is Back To Normal... He's ...,


In [10]:
movies_df.isnull().sum()

,0
id,0
title,6
genres,0
overview,954
vote_average,6
vote_count,6
tagline,25054
spoken_languages,0


In [12]:
movies_df.dropna(subset=['title'], inplace=True)


In [14]:
movies_df['overview'] = movies_df['overview'].fillna('')
movies_df['tagline'] = movies_df['tagline'].fillna('')


In [16]:
movies_df.isnull().sum()

,0
id,0
title,0
genres,0
overview,0
vote_average,0
vote_count,0
tagline,0
spoken_languages,0


In [20]:
movies_df["tags"] = movies_df['genres'] + " " + movies_df['overview'] + "" +  movies_df['tagline']

In [21]:
movies_df.head()

,id,title,genres,overview,vote_average,vote_count,tagline,spoken_languages,tags
0,862,Toy Story,,"Led by Woody, Andy's toys live happily in his ...",7.7,5415.0,,,"Led by Woody, Andy's toys live happily in his..."
1,8844,Jumanji,,When siblings Judy and Peter discover an encha...,6.9,2413.0,Roll the dice and unleash the excitement!,,When siblings Judy and Peter discover an ench...
2,15602,Grumpier Old Men,,A family wedding reignites the ancient feud be...,6.5,92.0,Still Yelling. Still Fighting. Still Ready for...,,A family wedding reignites the ancient feud b...
3,31357,Waiting to Exhale,,"Cheated on, mistreated and stepped on, the wom...",6.1,34.0,Friends are the people who let you be yourself...,,"Cheated on, mistreated and stepped on, the wo..."
4,11862,Father of the Bride Part II,,Just when George Banks has recovered from his ...,5.7,173.0,Just When His World Is Back To Normal... He's ...,,Just when George Banks has recovered from his...


In [23]:
print(movies_df['tags'].iloc[0])

 Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


In [28]:
import sys
!{sys.executable} -m pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

print("NLTK installed and 'stopwords', 'wordnet', 'punkt', and 'punkt_tab' corpora downloaded.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


NLTK installed and 'stopwords', 'wordnet', 'punkt', and 'punkt_tab' corpora downloaded.


In [30]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Initialize NLTK components
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[\W_]+', ' ', text) # Replaces non-alphanumeric with space
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords and lemmatize
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(processed_words)

# Apply the preprocessing function to the 'tags' column
movies_df['processed_tags'] = movies_df['tags'].apply(preprocess_text)

# Display the first few processed tags
print("First 5 entries of the 'processed_tags' column:")
print(movies_df['processed_tags'].head())

First 5 entries of the 'processed_tags' column:
0    led woody andy toy live happily room andy birt...
1    sibling judy peter discover enchanted board ga...
2    family wedding reignites ancient feud next doo...
3    cheated mistreated stepped woman holding breat...
4    george bank recovered daughter wedding receive...
Name: processed_tags, dtype: object


In [32]:
print(movies_df['processed_tags'].iloc[0])

led woody andy toy live happily room andy birthday brings buzz lightyear onto scene afraid losing place andy heart woody plot buzz circumstance separate buzz woody owner duo eventually learns put aside difference


In [34]:
movies_df.head()

,id,title,genres,overview,vote_average,vote_count,tagline,spoken_languages,tags,processed_tags
0,862,Toy Story,,"Led by Woody, Andy's toys live happily in his ...",7.7,5415.0,,,"Led by Woody, Andy's toys live happily in his...",led woody andy toy live happily room andy birt...
1,8844,Jumanji,,When siblings Judy and Peter discover an encha...,6.9,2413.0,Roll the dice and unleash the excitement!,,When siblings Judy and Peter discover an ench...,sibling judy peter discover enchanted board ga...
2,15602,Grumpier Old Men,,A family wedding reignites the ancient feud be...,6.5,92.0,Still Yelling. Still Fighting. Still Ready for...,,A family wedding reignites the ancient feud b...,family wedding reignites ancient feud next doo...
3,31357,Waiting to Exhale,,"Cheated on, mistreated and stepped on, the wom...",6.1,34.0,Friends are the people who let you be yourself...,,"Cheated on, mistreated and stepped on, the wo...",cheated mistreated stepped woman holding breat...
4,11862,Father of the Bride Part II,,Just when George Banks has recovered from his ...,5.7,173.0,Just When His World Is Back To Normal... He's ...,,Just when George Banks has recovered from his...,george bank recovered daughter wedding receive...


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
print("TfidfVectorizer imported successfully.")

TfidfVectorizer imported successfully.


In [40]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(1,2))
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['processed_tags'])
print(f"Shape of TF-IDF matrix: {tfidf_matrix.shape}")

Shape of TF-IDF matrix: (45460, 5000)


In [42]:
# Drop the original text columns and the intermediate 'tags' column
movies_df.drop(columns=['genres', 'overview', 'tagline', 'spoken_languages', 'tags'], inplace=True)

display(movies_df.head())

,id,title,vote_average,vote_count,processed_tags
0,862,Toy Story,7.7,5415.0,led woody andy toy live happily room andy birt...
1,8844,Jumanji,6.9,2413.0,sibling judy peter discover enchanted board ga...
2,15602,Grumpier Old Men,6.5,92.0,family wedding reignites ancient feud next doo...
3,31357,Waiting to Exhale,6.1,34.0,cheated mistreated stepped woman holding breat...
4,11862,Father of the Bride Part II,5.7,173.0,george bank recovered daughter wedding receive...


In [56]:
from sklearn.metrics.pairwise import cosine_similarity
print("cosine_similarity imported successfully.")

cosine_similarity imported successfully.


In [58]:
def recommend_movies(movie_title, tfidf_matrix, df):
    # Convert movie_title to lowercase for case-insensitive matching
    movie_title_lower = movie_title.lower()

    # Find the index of the movie in the DataFrame
    # Use .str.lower() on 'title' column for case-insensitive comparison
    movie_index = df[df['title'].str.lower() == movie_title_lower].index

    # Check if the movie was found
    if movie_index.empty:
        print(f"Movie '{movie_title}' not found in the dataset.")
        return pd.DataFrame(columns=['id', 'title'])

    # Get the index of the first matching movie (if multiple exist)
    idx = movie_index[0]

    # Calculate cosine similarities between the given movie and all other movies
    # Use tfidf_matrix[idx] as the reference vector
    cosine_sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    # Get the indices of the top 11 most similar movies (including itself)
    # We take 11 to exclude the movie itself later
    top_similar_indices = cosine_sim_scores.argsort()[-11:][::-1]

    # Exclude the movie itself and get the top 10 recommended movies
    # We need to make sure the original movie is not included in the recommendations
    recommended_movie_indices = [i for i in top_similar_indices if i != idx][:10]

    # Retrieve the 'id' and 'title' of the recommended movies
    recommended_movies_df = df.iloc[recommended_movie_indices][['id', 'title']]

    return recommended_movies_df.reset_index(drop=True)

# Test the function with 'Toy Story'
print("Recommendations for 'Toy Story':")
toy_story_recommendations = recommend_movies('Toy Story', tfidf_matrix, movies_df)
display(toy_story_recommendations)

# Test the function with 'Batman Begins'
print("\nRecommendations for 'Batman Begins':")
batman_begins_recommendations = recommend_movies('Batman Begins', tfidf_matrix, movies_df)
display(batman_begins_recommendations)

# Test the function with a non-existent movie
print("\nRecommendations for 'NonExistent Movie':")
non_existent_recommendations = recommend_movies('NonExistent Movie', tfidf_matrix, movies_df)
display(non_existent_recommendations)

Recommendations for 'Toy Story':


,id,title
0,10193,Toy Story 3
1,863,Toy Story 2
2,42816,The Champ
3,364123,Andy Kaufman Plays Carnegie Hall
4,92848,Andy Hardy's Blonde Trouble
5,6957,The 40 Year Old Virgin
6,355984,Andy Peters: Exclamation Mark Question Point
7,250434,Superstar: The Life and Times of Andy Warhol
8,1850,Man on the Moon
9,312399,Toy Reanimator



Recommendations for 'Batman Begins':


,id,title
0,29751,Batman Unmasked: The Psychology of the Dark Kn...
1,14919,Batman: Mask of the Phantasm
2,123025,"Batman: The Dark Knight Returns, Part 1"
3,414,Batman Forever
4,64202,Batman Beyond: The Movie
5,268,Batman
6,366924,Batman: Bad Blood
7,269246,Batman Beyond Darwyn Cooke's Batman 75th Anniv...
8,456348,Batman & Bill
9,300424,LEGO DC Comics Super Heroes: Batman: Be-Leaguered



Recommendations for 'NonExistent Movie':
Movie 'NonExistent Movie' not found in the dataset.


,id,title


In [59]:
import pickle

# Save the TF-IDF matrix
with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

# Save the movies DataFrame
with open('movies_df.pkl', 'wb') as f:
    pickle.dump(movies_df, f)

print("tfidf_matrix.pkl and movies_df.pkl saved successfully!")

tfidf_matrix.pkl and movies_df.pkl saved successfully!
